# Exploratory Analysis

## Eploring SpotiPy
- Explore available data
- Select features of interest
- Generate initial dataframe and database

###### Note:  Potentially building a recomender system that can take a set of the most frequently played songs from one user, and match them with a second users profile; potentially between courting couples and friends

In [1]:
# imports

import os
import sys
import json
import spotipy
import webbrowser
from os import getenv
import spotipy.util as util
from dotenv import load_dotenv
from json.decoder import JSONDecodeError
from spotipy.oauth2 import SpotifyClientCredentials

In [3]:
# We are using the client Module from the Python library for the Spotify API
# (https://spotipy.readthedocs.io/en/2.13.0/#module-spotipy.client)
# Client Credentials Flow


load_dotenv()  # this imports all .env variables

# Setting up env variables to connect to API
uri = getenv('uri') # must match in the Spotify app dashboard
SPOTIFY_CLIENT_ID = getenv('SPOTIFY_CLIENT_ID')
SPOTIFY_CLIENT_SECRET = getenv('SPOTIFY_CLIENT_SECRET')
username = getenv('USER_ID')  #  user who's data we are collecting
scope = 'user-top-read'  #  determines the kind of access you have to a user profile

# Access token to obtain user info
token = util.prompt_for_user_token(username=username,
                                   scope=scope,
                                   client_id=SPOTIFY_CLIENT_ID,
                                   client_secret=SPOTIFY_CLIENT_SECRET,
                                   redirect_uri=uri)

# activating spotify session
spotify_session = spotipy.Spotify(auth=token)

## Goals 
- Connect to user library using [scopes](https://developer.spotify.com/documentation/general/guides/scopes/)
- Scopes to connect to are [user-library-read](https://developer.spotify.com/documentation/general/guides/scopes/#user-library-read), [playlist-modify-public](https://developer.spotify.com/documentation/general/guides/scopes/#playlist-modify-public), and [user-top-read](https://developer.spotify.com/documentation/general/guides/scopes/#user-top-read)

##### *The goal here is to cnnect to the user's library, analyze it, and create a library.  One thing to keep in mind is that we will need user_1 and user_2 libraries*

In [4]:
# Playing with the api: accessing user top read, modify playlist, read library

# User top artists
top_artists = spotify_session.current_user_top_artists(limit=1)

# User top tracks
top_tracks = spotify_session.current_user_top_tracks(limit=1)

# Track_ids
top_tracks['items'][0]['id'] == "0akyEssGRVHstqCSWXusJL"
top_track_id = top_tracks['items'][0]['id']

# Track features
audio_anal = spotify_session.audio_analysis(top_track_id)

In [7]:
# Audio Analysis Objects

# See the following link for discriptive information on objects
# (https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-analysis/#time-interval-object)

# The following are one of three kinds of objects: 'Time interval'(TI), 'Sections'(Sc), or 'Segments'(Sg)
audio_anal['bars']   # (TI) Segment of time defined as given number of beats
audio_anal['beats']  # (TI) Time intervals of beats throughout track
audio_anal['sections']  # (Sc) Defined by large variations in rhythm
audio_anal['segments']  # (Sg) Song subdivisions, attempting to with each segment containing roughly a consistent sound
audio_anal['tatums']  # (TI) Beats are subdivisions of bars, and Tatums are subdivisions of beats


[{'start': 0.0,
  'duration': 0.80263,
  'confidence': 0.0,
  'loudness_start': -60.0,
  'loudness_max_time': 0.73422,
  'loudness_max': -58.563,
  'loudness_end': 0.0,
  'pitches': [0.467,
   0.241,
   0.261,
   0.24,
   0.357,
   0.363,
   0.383,
   0.378,
   0.321,
   0.347,
   1.0,
   0.984],
  'timbre': [0.168,
   172.317,
   11.834,
   -32.384,
   55.702,
   -49.92,
   15.295,
   5.387,
   -28.015,
   1.428,
   -9.548,
   -9.607]},
 {'start': 0.80263,
  'duration': 0.41061,
  'confidence': 1.0,
  'loudness_start': -59.263,
  'loudness_max_time': 0.00933,
  'loudness_max': -46.095,
  'loudness_end': 0.0,
  'pitches': [0.501,
   0.428,
   0.366,
   0.238,
   0.211,
   0.148,
   0.436,
   1.0,
   0.688,
   0.181,
   0.415,
   0.607],
  'timbre': [8.121,
   56.277,
   13.652,
   83.186,
   73.641,
   -74.075,
   -5.745,
   90.484,
   -29.97,
   -27.401,
   -2.698,
   -2.533]},
 {'start': 1.21324,
  'duration': 0.67909,
  'confidence': 1.0,
  'loudness_start': -58.877,
  'loudness_max

# Baseline Explorations
- Select type of problem type (class/reg)
- Determine model baselines
- Model evaluations/comparisons